In [ ]:
import torch, torchvision
print(f"PyTorch: {torch.__version__}, CUDA: {torch.version.cuda}")
print(f"Torchvision: {torchvision.__version__}")

PyTorch: 2.6.0+cu124, CUDA: 12.4
Torchvision: 0.21.0+cu124


In [1]:
# Install requirements
!git clone https://github.com/Tencent/Hunyuan3D-2.git
%cd Hunyuan3D-2
#!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118          #only install when get error in 1st cell
!pip install -r requirements.txt
!pip install -e .

# Install texture components with proper path handling
%cd hy3dgen/texgen/custom_rasterizer
!python3 setup.py install
%cd ../../../
%cd hy3dgen/texgen/differentiable_renderer
!python3 setup.py install
%cd ../../../

# Install requests for API calls
!pip install requests
!pip install rembg
!pip install onnxruntime


Cloning into 'Hunyuan3D-2'...
remote: Enumerating objects: 1043, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1043 (delta 15), reused 6 (delta 6), pack-reused 1011 (from 2)
Receiving objects: 100% (1043/1043), 81.26 MiB | 18.07 MiB/s, done.
Resolving deltas: 100% (488/488), done.
/content/Hunyuan3D-2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s

In [2]:
import sys
sys.path.append('/content/Hunyuan3D-2')

from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline
import torch
import requests
from PIL import Image
import numpy as np
from io import BytesIO
import time

In [3]:
# Initialize only the 3D shape pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained('tencent/Hunyuan3D-2', trust_remote_code=True)

2025-07-24 18:35:57,566 - hy3dgen.shapgen - INFO - Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2/hunyuan3d-dit-v2-0
INFO:hy3dgen.shapgen:Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2/hunyuan3d-dit-v2-0
2025-07-24 18:35:57,572 - hy3dgen.shapgen - INFO - Model path not exists, try to download from huggingface
INFO:hy3dgen.shapgen:Model path not exists, try to download from huggingface


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model.fp16.ckpt:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

2025-07-24 18:39:33,565 - hy3dgen.shapgen - INFO - Loading model from /root/.cache/huggingface/hub/models--tencent--Hunyuan3D-2/snapshots/34e28261f71c32975727be8db0eace439a280f82/hunyuan3d-dit-v2-0/model.fp16.safetensors
INFO:hy3dgen.shapgen:Loading model from /root/.cache/huggingface/hub/models--tencent--Hunyuan3D-2/snapshots/34e28261f71c32975727be8db0eace439a280f82/hunyuan3d-dit-v2-0/model.fp16.safetensors


PointCrossAttentionEncoder INFO: pc_sharpedge_size is given, using pc_size=5120, pc_sharpedge_size=5120


In [4]:
import gc
from google.colab import userdata
from rembg import remove
import cv2
#load your hf token from secrets or
#HF_TOKEN ="paste here token"
HF_TOKEN = userdata.get('HF_TOKEN')
print("HF Token loaded from Colab secrets")


HF Token loaded from Colab secrets


In [5]:
# Use best model for image generation
API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"

def remove_background(image):


    # Convert PIL to bytes
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    # Remove background
    output = remove(img_byte_arr)

    result_image = Image.open(BytesIO(output))
    white_bg = Image.new('RGB', result_image.size, (255, 255, 255))
    if result_image.mode == 'RGBA':
        white_bg.paste(result_image, mask=result_image.split()[3])
    else:
        white_bg.paste(result_image)

    return white_bg

def generate_image_from_text(prompt, max_retries=3):
    headers = {}
    if HF_TOKEN:
        headers = {"Authorization": f"Bearer {HF_TOKEN}"}

    # Enhanced prompt engineering for better 3D generation
    enhanced_prompt = f"{prompt}, highly detailed, professional product photography, studio lighting, white background, single object, clean, sharp focus, 8k resolution, photorealistic, centered composition, no shadows, isolated object"

    payload = {
        "inputs": enhanced_prompt,
        "parameters": {
            "height": 512,
            "width": 512,
            "num_inference_steps": 28,
            "guidance_scale": 3.5
        }
    }

    for attempt in range(max_retries):
        print(f"Generating image from text (attempt {attempt + 1}/{max_retries}): {prompt}")

        response = requests.post(API_URL, headers=headers, json=payload)

        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))

            # Remove background for better 3D results
            print("Removing background...")
            image = remove_background(image)

            image.save("generated_image.png")
            print("Image generated and background removed successfully!")
            return image
        elif response.status_code == 503:
            print("Model is loading, waiting 20 seconds...")
            time.sleep(20)
        else:
            print(f"Error: {response.status_code}, {response.text}")
            if attempt < max_retries - 1:
                print(f"Retrying in 5 seconds...")
                time.sleep(5)

    raise Exception("Failed to generate image after multiple attempts")

def clear_memory():

    # Clear shape pipeline from memory
    global shape_pipeline
    del shape_pipeline

    gc.collect()
    torch.cuda.empty_cache()




In [6]:
def text_to_3d_pipeline(prompt):


    # Generate image
    image = generate_image_from_text(prompt)

    # Generate 3D
    with torch.no_grad():
        mesh = shape_pipeline(image=image)[0]

    mesh.export("output_3d_model.glb")

    # generate_texture = input("Generate texture? (y/n): ").lower().strip()

    # if generate_texture in ['y', 'yes']:
    #     textured_mesh = generate_texture_for_glb(
    #         "output_3d_model.glb",
    #         "generated_image.png",
    #         prompt
    #     )
    #     return textured_mesh, mesh, image
    # else:
    return mesh, image

def image_to_3d_pipeline(image_path, prompt=""):


    # Load and process image
    image = Image.open(image_path) if isinstance(image_path, str) else image_path
    image = remove_background(image)
    image.save("processed_image.png")

    # Generate 3D from image
    with torch.no_grad():
        mesh = shape_pipeline(image=image)[0]

    mesh.export("output_3d_model.glb")

    # Ask for texture generation
    # generate_texture = input("Generate texture? (y/n): ").lower().strip()

    # if generate_texture in ['y', 'yes']:
    #     if not prompt:
    #         prompt = input("Enter texture description: ")

    #     textured_mesh = generate_texture_for_glb(
    #         "output_3d_model.glb",
    #         "processed_image.png",
    #         prompt
    #     )
    #     return textured_mesh, mesh, image
    # else:
    return mesh, image


In [7]:
result  = text_to_3d_pipeline('a big scifi tank ')

Generating image from text (attempt 1/3): a big scifi tank 


Removing background...


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 185GB/s]


Image generated and background removed successfully!


Volume Decoding: 100%|██████████| 7134/7134 [02:23<00:00, 49.84it/s]


Generate texture? (y/n): n


In [ ]:
import torch
import gc
from PIL import Image
import trimesh
import numpy as np

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def generate_texture_for_glb(glb_path, png_path, prompt=""):
    """Generate texture for GLB file using Realistic Vision v6 - Simple and effective"""

    clear_memory()

    # Load mesh with error handling
    try:
        loaded = trimesh.load(glb_path)
        if isinstance(loaded, trimesh.Scene):
            mesh = list(loaded.geometry.values())[0]
        else:
            mesh = loaded
    except Exception as e:
        print(f"GLB loading error: {e}")
        return None

    # Load and process image
    image = Image.open(png_path)
    if image.size[0] > 512 or image.size[1] > 512:
        image = image.resize((512, 512), Image.Resampling.LANCZOS)

    try:
        print("Generating texture with Realistic Vision v6...")
        from diffusers import StableDiffusionPipeline

        # Use Realistic Vision v6 - best for photorealistic textures in 2025
        pipe = StableDiffusionPipeline.from_pretrained(
            "SG161222/Realistic_Vision_V6.0_B1_noVAE",
            torch_dtype=torch.float16,
            safety_checker=None,
            requires_safety_checker=False
        )

        if torch.cuda.is_available():
            pipe = pipe.to("cuda")
            pipe.enable_attention_slicing()


        # Simple effective prompt
        texture_prompt = f"seamless {prompt} texture pattern, high quality, detailed material surface" if prompt else "seamless texture pattern, high quality material"

        with torch.no_grad():
            texture_image = pipe(
                texture_prompt,
                num_inference_steps=25,
                height=512,
                width=512,
                guidance_scale=7.5,
                negative_prompt="blurry, low quality, seams, tiled, repeating pattern"
            ).images[0]

        del pipe
        clear_memory()

        print("Texture generated successfully!")

    except Exception as e:
        print(f"Texture generation failed: {e}")
        print("Using enhanced original image...")
        texture_image = enhance_original_image(image)

    try:
        material = trimesh.visual.material.PBRMaterial(
            baseColorTexture=texture_image,
            metallicFactor=0.0,
            roughnessFactor=0.8
        )
        mesh.visual.material = material


        output_path = "textured_3d_model.glb"
        mesh.export(output_path)
        print(f"Textured model saved to {output_path}")

    except Exception as e:
        print(f"Error applying texture: {e}")

    return mesh

def enhance_original_image(image):
    """Enhance the original image as fallback"""
    img_array = np.array(image)

    # Simple enhancement
    enhanced = img_array.astype(float)
    enhanced = (enhanced - 128) * 1.2 + 128  # Contrast

    # Add texture noise
    noise = np.random.normal(0, 8, img_array.shape)
    enhanced += noise

    return Image.fromarray(np.clip(enhanced, 0, 255).astype(np.uint8))



🎨 Texture Generator 2025 Ready!
Usage: generate_texture_for_glb('model.glb', 'image.png', 'wood texture')
Powered by Realistic Vision v6 - Best texture model for 2025


In [ ]:
generate_texture_for_glb("/content/output_3d_model (2).glb","/content/Tree (2).png")

Generating texture with Realistic Vision v6...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/hug

  0%|          | 0/25 [00:00<?, ?it/s]

Texture generated successfully!
Textured model saved to textured_3d_model.glb


<trimesh.Trimesh(vertices.shape=(501414, 3), faces.shape=(1002724, 3))>